In [2]:
import torch
from diffusers import StableDiffusionPipeline

# 加载 Stable Diffusion 1.5 模型
pipe = StableDiffusionPipeline.from_pretrained("/media/allenyljiang/5234E69834E67DFB/StableDiffusion_Models/stable-diffusion-v1-5")
pipe = pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


### styleID 处理方式

In [4]:
from styleid_utils import get_unet_layers
unet = pipe.unet
# Get residual and attention block in decoder
# [0 ~ 11], total 12 layers
resnet, attn = get_unet_layers(unet) # 明确知道模块中哪些层有attention

In [7]:
print(len(resnet),len(attn))

12 12


In [17]:
def print_real_attention_layer_names(attn_layers, unet, qkv_injection_layer_num):
    attn_layer_names = []
    
    # 获取所有模块的名字和对象
    named_modules = dict(unet.named_modules())
    
    # 遍历指定的 qkv_injection_layer_num
    for idx in qkv_injection_layer_num:
        if attn_layers[idx] is not None:  # 确保这个索引有 Attention 层
            attn_layer = attn_layers[idx]
            
            # 根据 Attention 层对象找到对应的名称
            for name, module in named_modules.items():
                if module is attn_layer:
                    attn_layer_names.append(name)
                    break
    
    # 打印结果
    print("Attention Layers:", attn_layer_names)

# 示例调用
unet = pipe.unet
resnet_layers, attn_layers = get_unet_layers(unet)

# 定义 qkv_injection_layer_num
qkv_injection_layer_num = [7, 8, 9, 10, 11]

# 调用函数打印 Attention 层名
print_real_attention_layer_names(attn_layers, unet, qkv_injection_layer_num)


Attention Layers: ['up_blocks.2.attentions.1', 'up_blocks.2.attentions.2', 'up_blocks.3.attentions.0', 'up_blocks.3.attentions.1', 'up_blocks.3.attentions.2']


In [25]:
attn[3].transformer_blocks[0].attn1

Attention(
  (to_q): Linear(in_features=1280, out_features=1280, bias=False)
  (to_k): Linear(in_features=1280, out_features=1280, bias=False)
  (to_v): Linear(in_features=1280, out_features=1280, bias=False)
  (to_out): ModuleList(
    (0): Linear(in_features=1280, out_features=1280, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )
)

In [28]:
qkv_injection_layer_num = [7, 8, 9, 10, 11]
attn_features = {}
for i in qkv_injection_layer_num:
    attn_features["layer{}_attn".format(i)] = {}
    print(attn[i].transformer_blocks[0].attn1)
#     attn[i].transformer_blocks[0].attn1.register_forward_hook(self.__get_query_key_value("layer{}_attn".format(i)))

# # Modify hook (if you change query key value)
# for i in qkv_injection_layer_num:
#     attn[i].transformer_blocks[0].attn1.register_forward_hook(self.__modify_self_attn_qkv("layer{}_attn".format(i)))


Attention(
  (to_q): Linear(in_features=640, out_features=640, bias=False)
  (to_k): Linear(in_features=640, out_features=640, bias=False)
  (to_v): Linear(in_features=640, out_features=640, bias=False)
  (to_out): ModuleList(
    (0): Linear(in_features=640, out_features=640, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )
)
Attention(
  (to_q): Linear(in_features=640, out_features=640, bias=False)
  (to_k): Linear(in_features=640, out_features=640, bias=False)
  (to_v): Linear(in_features=640, out_features=640, bias=False)
  (to_out): ModuleList(
    (0): Linear(in_features=640, out_features=640, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )
)
Attention(
  (to_q): Linear(in_features=320, out_features=320, bias=False)
  (to_k): Linear(in_features=320, out_features=320, bias=False)
  (to_v): Linear(in_features=320, out_features=320, bias=False)
  (to_out): ModuleList(
    (0): Linear(in_features=320, out_features=320, bias=True)
    (1): Dropout(p=0.0, inplace=False)
  )


In [29]:
def get_attention_layer_names(attn_layers, qkv_injection_layer_num, unet):
    attn_layer_names = []
    
    # 获取所有命名的模块
    named_modules = dict(unet.named_modules())
    
    # 遍历 qkv_injection_layer_num 列表
    for idx in qkv_injection_layer_num:
        # 访问 attn[i] 的 transformer_blocks[0].attn1
        attn_layer = attn_layers[idx].transformer_blocks[0].attn1
        
        # 在所有模块中找到该层的名字
        for name, module in named_modules.items():
            if module is attn_layer:
                attn_layer_names.append(name)
                break

    # 打印所有 Attention 层的名字
    for name in attn_layer_names:
        print(name)

# 假设 unet 模型已经加载
unet = pipe.unet
resnet_layers, attn_layers = get_unet_layers(unet)

# 定义 qkv_injection_layer_num
qkv_injection_layer_num = [7, 8, 9, 10, 11]

# 打印这些 attention 层的名字
get_attention_layer_names(attn_layers, qkv_injection_layer_num, unet)


up_blocks.2.attentions.1.transformer_blocks.0.attn1
up_blocks.2.attentions.2.transformer_blocks.0.attn1
up_blocks.3.attentions.0.transformer_blocks.0.attn1
up_blocks.3.attentions.1.transformer_blocks.0.attn1
up_blocks.3.attentions.2.transformer_blocks.0.attn1


In [13]:
def find_attention_layers(model):
    # 递归查找 Attention 层
    attention_layers = []
    for name, module in model.named_modules():
        if 'Attention' in str(type(module)):  # 检查层类型是否为 Attention
            attention_layers.append(name)
    return attention_layers

# 获取 U-Net 模型中的 Attention 层
attention_layers = find_attention_layers(pipe.unet)

# 打印出所有包含 Attention 层的名称
print("Attention Layers:", attention_layers) # sd1.5总共32层


Attention Layers: ['down_blocks.0.attentions.0.transformer_blocks.0.attn1', 'down_blocks.0.attentions.0.transformer_blocks.0.attn2', 'down_blocks.0.attentions.1.transformer_blocks.0.attn1', 'down_blocks.0.attentions.1.transformer_blocks.0.attn2', 'down_blocks.1.attentions.0.transformer_blocks.0.attn1', 'down_blocks.1.attentions.0.transformer_blocks.0.attn2', 'down_blocks.1.attentions.1.transformer_blocks.0.attn1', 'down_blocks.1.attentions.1.transformer_blocks.0.attn2', 'down_blocks.2.attentions.0.transformer_blocks.0.attn1', 'down_blocks.2.attentions.0.transformer_blocks.0.attn2', 'down_blocks.2.attentions.1.transformer_blocks.0.attn1', 'down_blocks.2.attentions.1.transformer_blocks.0.attn2', 'up_blocks.1.attentions.0.transformer_blocks.0.attn1', 'up_blocks.1.attentions.0.transformer_blocks.0.attn2', 'up_blocks.1.attentions.1.transformer_blocks.0.attn1', 'up_blocks.1.attentions.1.transformer_blocks.0.attn2', 'up_blocks.1.attentions.2.transformer_blocks.0.attn1', 'up_blocks.1.attention

### cross image attention方式

In [17]:
def register_recr(net_,name, count, place_in_unet):
    '''
    在这里实现指定层添加自定义的AttentionProcessor
    '''
    if net_.__class__.__name__ == 'ResnetBlock2D':
        pass
    if net_.__class__.__name__ == 'Attention':
        if name.endswith("attn1"):
            if place_in_unet == "down": 
                down_layers.append(name)
            elif place_in_unet == "mid":
                middle_layers.append(name)
            elif place_in_unet == "up":
                up_layers.append(name)
        # net_.set_processor(AttentionProcessor(place_in_unet + f"_{count + 1}"))
        return count + 1
    elif hasattr(net_, 'children'):
        for child_name,net__ in net_.named_children():
            new_full_name = f"{name}.{child_name}" if name else child_name
            count = register_recr(net__, new_full_name,count, place_in_unet)
    return count

down_layers = []
middle_layers = []
up_layers = []
cross_att_count = 0
sub_nets = unet.named_children()
for net_name,net in sub_nets:
    if "down" in net_name:
        cross_att_count += register_recr(net,net_name, 0, "down")
    elif "up" in net_name:
        cross_att_count += register_recr(net,net_name, 0, "up")
    elif "mid" in net_name:
        cross_att_count += register_recr(net,net_name, 0, "mid")
# print(list([sub[0] for sub in sub_nets]))

print("down layers",len(down_layers),"up_layers",len(up_layers),"middle layers",len(middle_layers))

down layers 6 up_layers 9 middle layers 1


In [18]:
up_layers  # [4:10]

['up_blocks.1.attentions.0.transformer_blocks.0.attn1',
 'up_blocks.1.attentions.1.transformer_blocks.0.attn1',
 'up_blocks.1.attentions.2.transformer_blocks.0.attn1',
 'up_blocks.2.attentions.0.transformer_blocks.0.attn1',
 'up_blocks.2.attentions.1.transformer_blocks.0.attn1',
 'up_blocks.2.attentions.2.transformer_blocks.0.attn1',
 'up_blocks.3.attentions.0.transformer_blocks.0.attn1',
 'up_blocks.3.attentions.1.transformer_blocks.0.attn1',
 'up_blocks.3.attentions.2.transformer_blocks.0.attn1']

: 